In [1]:
%load_ext autoreload
%autoreload 2

import logging
import torch
import pytorch_lightning as pl
import os
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T
from ptls.preprocessing import PysparkDataPreprocessor


libgomp: Invalid value for environment variable OMP_NUM_THREADS

libgomp: Invalid value for environment variable OMP_NUM_THREADS


In [3]:
import os
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T

In [ ]:
data_path = 'data/'

spark_conf = pyspark.SparkConf()
spark_conf.setMaster("local[*]").setAppName("PysparkDataPreprocessor")
spark_conf.set("spark.driver.maxResultSize", "4g")
spark_conf.set("spark.executor.memory", "16g")
spark_conf.set("spark.executor.memoryOverhead", "4g")
spark_conf.set("spark.driver.memory", "16g")
spark_conf.set("spark.driver.memoryOverhead", "4g")
spark_conf.set("spark.cores.max", "24")
spark_conf.set("spark.sql.shuffle.partitions", "200")
spark_conf.set("spark.local.dir", "../../spark_local_dir")


spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
spark.sparkContext.getConf().getAll()

# Dial

In [ ]:
dial_train = spark.read.parquet('dial_train.parquet')
dial_test = spark.read.parquet('dial_test.parquet')

# Targets

In [ ]:
train_target = spark.read.parquet('train_target.parquet')

In [ ]:
preprocessor_target = PysparkDataPreprocessor(
    col_id="client_id",
    col_event_time="mon",
    event_time_transformation="dt_to_timestamp",
    cols_identity=["target_1", "target_2", "target_3", "target_4"],
)
processed_target = preprocessor_target.fit_transform(train_target).drop('event_time')

In [ ]:
processed_train = processed_train.join(processed_target, processed_target.client_id == processed_train.client_id, 'left').drop(processed_target.client_id)

In [ ]:
processed_train.write.parquet('dial_train_prepr.parquet', mode='overwrite')
processed_test.write.parquet('dial_test_prepr.parquet', mode='overwrite')